In [1]:
# | default_exp tools.task_planner

In [7]:
# | export
"""
Task Planner BaseTool - Complete Task Analyzer Integration with Caching
"""
import json
import hashlib
import os
from typing import Dict, Any
from agentic.tools.base import BaseTool
from agentic.analyzer import AgentTaskAnalyzer

class TaskPlannerTool(BaseTool):
    def __init__(self):
        super().__init__()
        self.cache_dir = "/tmp/buddy_cache"
        os.makedirs(self.cache_dir, exist_ok=True)
    
    def _get_cache_key(self, request: str) -> str:
        """Generate cache key from request"""
        return hashlib.md5(request.encode()).hexdigest()
    
    def _get_cache_path(self, cache_key: str) -> str:
        """Get cache file path"""
        return os.path.join(self.cache_dir, f"task_plan_{cache_key}.json")
    
    def _load_from_cache(self, request: str) -> Dict[str, Any]:
        """Load cached plan if exists"""
        cache_key = self._get_cache_key(request)
        cache_path = self._get_cache_path(cache_key)
        
        if os.path.exists(cache_path):
            try:
                with open(cache_path, 'r') as f:
                    cached_result = json.load(f)
                print(f"📦 Loaded cached plan from {cache_path}")
                return cached_result
            except Exception as e:
                print(f"⚠️  Cache load failed: {e}")
        
        return None
    
    def _save_to_cache(self, request: str, result: Dict[str, Any]) -> None:
        """Save plan to cache"""
        cache_key = self._get_cache_key(request)
        cache_path = self._get_cache_path(cache_key)
        
        try:
            with open(cache_path, 'w') as f:
                json.dump(result, f, indent=2)
            print(f"💾 Cached plan to {cache_path}")
        except Exception as e:
            print(f"⚠️  Cache save failed: {e}")
    def get_parameters_schema(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "task_planner",
                "description": "PRIORITY TOOL: Analyze complex requests and create detailed execution plans with task breakdown, framework selection via debate, and success criteria. ALWAYS pass the complete user input, not summaries. Has ABSOLUTE PRECEDENCE over other tools for moderate to complex requests. Creates debate tasks instead of calling debate_agent directly.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "request": {
                            "type": "string",
                            "description": "The complete user request to analyze and plan (never use summaries)"
                        },
                        "model": {
                            "type": "string", 
                            "description": "LLM model for analysis",
                            "default": "qwen3:14b"
                        },
                        "base_url": {
                            "type": "string",
                            "description": "LLM base URL",
                            "default": "http://192.168.29.147:11500/v1"
                        }
                    },
                    "required": ["request"]
                }
            }
        }
    
    def _needs_elaboration(self, request: str) -> bool:
        """Check if request needs more details"""
        # Simple heuristics to detect vague requests
        vague_indicators = [
            len(request.split()) < 5,  # Very short requests
            any(word in request.lower() for word in ['something', 'anything', 'stuff', 'thing']),
            request.count('?') > 2,  # Too many questions
            not any(word in request.lower() for word in ['create', 'build', 'make', 'develop', 'implement', 'setup', 'configure'])
        ]
        return any(vague_indicators)

    def _ask_for_elaboration(self, request: str) -> Dict[str, Any]:
        """Ask user to elaborate on vague request"""
        return {
            "success": False,
            "status": "needs_elaboration",
            "message": "I need more details to create an effective plan. Could you please elaborate on:",
            "clarification_needed": [
                "What specific outcome do you want to achieve?",
                "What technologies or frameworks should be used?",
                "Are there any constraints or requirements I should know about?",
                "What is the scope and complexity level you're aiming for?"
            ],
            "original_request": request,
            "suggestion": "Please provide a more detailed description of what you want to accomplish, and I'll create a comprehensive execution plan for you."
        }

    def execute(self, request: str, model: str = "qwen3:14b", base_url: str = "http://192.168.29.147:11500/v1") -> Dict[str, Any]:
        """Execute complete task planning with elaboration check - accepts full user input"""
        try:
            # Check if request needs elaboration
            if self._needs_elaboration(request):
                return self._ask_for_elaboration(request)
            
            # Check cache first
            cached_result = self._load_from_cache(request)
            if cached_result:
                return cached_result
            
            # Initialize with full analyzer capabilities
            analyzer = AgentTaskAnalyzer(model=model, base_url=base_url)
            
            # Get complete analysis using the full user request
            result = analyzer.analyze(request)
            
            # Display beautiful analysis (this will show in Buddy output)
            analyzer.display_result(result)
            
            # Return structured plan for Buddy to use
            plan_result = {
                "success": True,
                "status": "success",
                "analysis_complete": True,
                "complexity": result.input_complexity.value,
                "total_tasks": len(result.tasks),
                "total_actions": result.execution_plan.total_actions,
                "frameworks_selected": result.recommended_frameworks,
                "tools_needed": [tool.value for tool in result.buddy_tools_needed],
                "execution_plan": {
                    "sequential_phases": result.execution_plan.sequential_phases,
                    "parallel_groups": result.execution_plan.parallel_groups,
                    "critical_path": result.execution_plan.critical_path
                },
                "tasks": [
                    {
                        "id": task.id,
                        "name": task.name,
                        "description": task.description,
                        "complexity": task.complexity.value,
                        "tools": [tool.value for tool in task.buddy_tools],
                        "frameworks": task.frameworks,
                        "actions": task.actions,
                        "success_criteria": task.success_criteria,
                        "dependencies": task.dependencies,
                        "expected_outputs": task.expected_outputs,
                        "execution_mode": task.execution_mode.value
                    }
                    for task in result.tasks
                ],
                "success_criteria": result.success_criteria,
                "plan_summary": f"✅ Created {len(result.tasks)} tasks with {result.execution_plan.total_actions} actions. Complexity: {result.input_complexity.value}. Ready for execution!",
                "next_steps": "Execute tasks in the order specified in execution_plan.sequential_phases using the recommended tools.",
                "original_request": request,  # Store original request for task_executor
                "requires_execution": True  # Flag to trigger automatic task_executor
            }
            
            # Cache the result
            self._save_to_cache(request, plan_result)
            
            return plan_result
            
        except Exception as e:
            return {
                "success": False,
                "status": "error",
                "error": str(e),
                "plan_summary": f"❌ Failed to create execution plan: {e}",
                "next_steps": "Try breaking down the request into simpler parts or check the request format."
            }


ImportError: cannot import name 'AgentTaskAnalyzer' from 'agentic.analyzer' (/home/pranav-pc/projects/applied-GenAI-lab/agentic/backend/analyzer.py)